In [14]:
%matplotlib inline
import pymongo
import networkx as nx
import time
import itertools
import operator

from multiprocessing import Pool
from pymongo import MongoClient

from sweepy.get_config import get_config

config = get_config()

MONGO_URL = config.get('MONGO_URL')
MONGO_PORT = config.get('MONGO_PORT')
MONGO_USERNAME = config.get('MONGO_USERNAME')
MONGO_PASSWORD = config.get('MONGO_PASSWORD')

client = MongoClient(MONGO_URL, int(MONGO_PORT))

db = client.tweets
db.authenticate(MONGO_USERNAME, MONGO_PASSWORD)

users = db.users
all_users = users.find()

In [2]:
graph  = nx.DiGraph()

In [3]:
for user in all_users:
    graph.add_node(str(user['id_str']))
    for friend_id in user['friends_ids']:
        if not friend_id in graph:
            graph.add_node(str(friend_id))
        graph.add_edge(str(user['id_str']), str(friend_id))

In [9]:
print nx.info(graph)

Name: 
Type: DiGraph
Number of nodes: 2419120
Number of edges: 5248563
Average in degree:   2.1696
Average out degree:   2.1696


In [4]:
eigen_values = nx.eigenvector_centrality(graph)

In [10]:
sorted_eigen_values = sorted(eigen_values.items(), key=operator.itemgetter(1), reverse=True)
sorted_eigen_values[:10]

[('121817564', 0.09428345177961117),
 ('11107172', 0.07497270265786563),
 ('807095', 0.06812954689012012),
 ('216695058', 0.06667872003411948),
 ('20861943', 0.06605647185989534),
 ('58524428', 0.06489206418816294),
 ('16104258', 0.06282196653973698),
 ('18816166', 0.06144353212620821),
 ('436873679', 0.05926741271555719),
 ('14677919', 0.0592280032725243)]

In [4]:
closeness_values = nx.closeness_centrality(graph)

KeyboardInterrupt: 

In [ ]:
sorted_closeness_values = sorted(closeness_values.items(), key=operator.itemgetter(1), reverse=True)
sorted_closeness_values[:10]

#iGraph

In [10]:
from igraph import *

In [17]:
g = Graph(directed=True)
all_users = users.find()

In [18]:
names = []
edges = []
for user in all_users:
    names.append(str(user['id_str']))
    g.add_vertices(1)
    for friend_id in user['friends_ids']:
        edges.append((user['id_str'], str(friend_id)))
        try:
            g.vs.find(name=str(friend_id))
        except ValueError:
            names.append(str(friend_id))
            g.add_vertices(1)
    
g.vs['name'] = names
g.add_edges(edges)

In [19]:
print g.summary()

IGRAPH DN-- 5253080 5248564 -- 
+ attr: name (v)


In [56]:
thisisfusion = g.vs.find(name='121817564')
thisisfusion.degree(type="out")

1907

In [ ]:
ebs = g.edge_betweenness()
max_eb = max(ebs)
[g.es[idx].tuple for idx, eb in enumerate(ebs) if eb == max_eb]